In [1]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")



D=8;
chi=40;



J1=0.80902;
J2=0;
J3=0;
Jchi=0;
Jtrip=0.5878;

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);


In [2]:
function read_json_state(filenm)
    json_dict = Dict()
    open(filenm, "r") do f
        json_dict
        dicttxt = read(f,String)  # file information to string
        json_dict=JSON.parse(dicttxt)  # parse and transform data
    end
    return json_dict
end



read_json_state (generic function with 1 method)

In [3]:
function wrap_json_state(Bond_irrep,Bond_A_coe,Bond_B_coe,Triangle_A1_coe,Triangle_A2_coe)
    if Bond_irrep=="A"
        coes=Dict([("Bond_A_coe", create_coe_dict(Bond_A_coe)), ("Triangle_A1_coe", create_coe_dict(Triangle_A1_coe)),("Triangle_A2_coe", create_coe_dict(Triangle_A2_coe))]);
    elseif Bond_irrep=="B"
        coes=Dict([("Bond_B_coe", create_coe_dict(Bond_B_coe)), ("Triangle_A1_coe", create_coe_dict(Triangle_A1_coe)),("Triangle_A2_coe", create_coe_dict(Triangle_A2_coe))]);
    elseif Bond_irrep=="A+iB"
        coes=Dict([("Bond_A_coe", create_coe_dict(Bond_A_coe)), ("Bond_B_coe", create_coe_dict(Bond_B_coe)), ("Triangle_A1_coe", create_coe_dict(Triangle_A1_coe)),("Triangle_A2_coe", create_coe_dict(Triangle_A2_coe))]);
    end
    json_state=Dict([("coes" , coes), ("Bond_irrep", Bond_irrep)]);
    return json_state
end
function create_coe_dict(coe)
    #print(coe)
    entries=Vector(undef,length(coe));
    for cc=1:length(coe)
        entries[cc]=string(cc-1)*" "*string(coe[cc]);
    end
    dims=Vector(undef,1);
    dims[1]=length(coe);

    coe_dict=Dict([("dtype", "float64"), ("numEntries", length(coe)),("entries", entries), ("dims", dims)]);
    return coe_dict
end

create_coe_dict (generic function with 1 method)

In [4]:
function initial_state(Bond_irrep,D,init_statenm=nothing)
    if init_statenm==nothing 
        println("Random initial state");flush(stdout);
        A_set,B_set,A1_set,A2_set, _,_,_,_, _, _, _, _, _=construct_tensor(D);
        if Bond_irrep=="A"
            Bond_A_coe=randn(Float64, length(A_set));
            Bond_B_coe=[];
        elseif Bond_irrep=="B"
            Bond_A_coe=[];
            Bond_B_coe=randn(Float64, length(B_set));
        elseif Bond_irrep=="A+iB"
            Bond_A_coe=randn(Float64, length(A_set));
            Bond_B_coe=randn(Float64, length(B_set));
        end
        Triangle_A1_coe=randn(Float64, length(A1_set));
        Triangle_A2_coe=randn(Float64, length(A2_set));
        
        json_state_dict=wrap_json_state(Bond_irrep, Bond_A_coe, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe)
    else
        println("load state: "*init_statenm);flush(stdout);
        json_state_dict=read_json_state(init_statenm);
        Bond_irrep_, Bond_A_coe, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe=get_tensor_coes(json_dict)
        @assert Bond_irrep_==Bond_irrep
    end
    return json_state_dict, Bond_A_coe, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe

end


initial_state (generic function with 2 methods)

In [5]:
function energy_CTM(D,chi,parameters, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol,state_dict)

    A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);
    
    bond_tensor,triangle_tensor=construct_su2_PG_IPESS(state_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);
    
    PEPS_tensor=bond_tensor;
    @tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
    A_unfused=PEPS_tensor;
    
    U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
    @tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];

    CTM=[];
    U_L=[];
    U_D=[];
    U_R=[];
    U_U=[];

    init=Dict([("CTM", []), ("init_type", "PBC")]);
    conv_check="singular_value";
    CTM_ite_info=false;
    CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,CTM_conv_tol,init,CTM_ite_nums,CTM_trun_tol,CTM_ite_info);
    
    E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_triangle");
    #E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_bond");
    energy=(E_up+E_down)/3;

    #return energy,CTM,U_L,U_D,U_R,U_U
    return energy
end



energy_CTM (generic function with 1 method)

In [6]:
function get_vector(json_dict)
    Bond_irrep, Bond_A_coe, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe=get_tensor_coes(json_dict);
    if Bond_irrep=="A"
        vec=vcat(Bond_A_coe,Triangle_A1_coe,Triangle_A2_coe);
    elseif Bond_irrep=="B"
        vec=vcat(Bond_B_coe,Triangle_A1_coe,Triangle_A2_coe);
    elseif Bond_irrep=="A+iB"
        vec=vcat(Bond_A_coe,Bond_B_coe,Triangle_A1_coe,Triangle_A2_coe);
    end
    return vec
end


get_vector (generic function with 1 method)

In [7]:
function set_vector(json_dict, vec)
    Bond_irrep, Bond_A_coes0, Bond_B_coes0, Triangle_A1_coes0, Triangle_A2_coes0=get_tensor_coes(json_dict);
    if Bond_irrep=="A"
        siz=length(Bond_A_coes0)
        Bond_A_coe=vec[1:siz]
        vec=vec[siz+1:length(vec)]
        siz=length(Triangle_A1_coes0)
        Triangle_A1_coe=vec[1:siz]
        vec=vec[siz+1:length(vec)]
        siz=length(Triangle_A2_coes0)
        Triangle_A2_coe=vec[1:siz]

        Bond_B_coe=nothing;
    elseif Bond_irrep=="B"
        siz=length(Bond_B_coes0)
        Bond_B_coe=vec[1:siz]
        vec=vec[siz+1:length(vec)]
        siz=length(Triangle_A1_coes0)
        Triangle_A1_coe=vec[1:siz]
        vec=vec[siz+1:length(vec)]
        siz=length(Triangle_A2_coes0)
        Triangle_A2_coe=vec[1:siz]

        Bond_A_coe=nothing;
    elseif Bond_irrep=="A+iB"
        siz=length(Bond_A_coes0)
        Bond_A_coe=vec[1:siz]
        vec=vec[siz+1:length(vec)]
        siz=length(Bond_B_coes0)
        Bond_B_coe=vec[1:siz]
        vec=vec[siz+1:length(vec)]
        siz=length(Triangle_A1_coes0)
        Triangle_A1_coe=vec[1:siz]
        vec=vec[siz+1:length(vec)]
        siz=length(Triangle_A2_coes0)
        Triangle_A2_coe=vec[1:siz]

    end
    json_dict_new=wrap_json_state(Bond_irrep, Bond_A_coe, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe)
    #return Bond_irrep, Bond_A_coe, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe
    return json_dict_new
end

set_vector (generic function with 1 method)

In [8]:


function normalize_IPESS_SU2_PG(state_dict)
    Bond_irrep, Bond_A_coe, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe=get_tensor_coes(state_dict)
    if Bond_irrep=="A"
        Bond_norm=norm(Bond_A_coe)
        Bond_A_coe=Bond_A_coe/Bond_norm
    elseif Bond_irrep=="B"
        Bond_norm=norm(Bond_B_coe)
        Bond_B_coe=Bond_B_coe/Bond_norm
    elseif state.Bond_irrep=="A+iB"
        Bond_norm=sqrt(norm(Bond_A_coe)^2+norm(Bond_B_coe)^2)
        Bond_A_coe=Bond_A_coe/Bond_norm
        Bond_B_coe=Bond_B_coe/Bond_norm
    end
    Triangle_norm=sqrt(norm(Triangle_A1_coe)^2+norm(Triangle_A2_coe)^2)
    Triangle_A1_coe=Triangle_A1_coe/Triangle_norm
    Triangle_A2_coe=Triangle_A2_coe/Triangle_norm

    state_dict=wrap_json_state(Bond_irrep, Bond_A_coe, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe)
    return state_dict
end


normalize_IPESS_SU2_PG (generic function with 1 method)

In [9]:

function Grad_FiniteDiff(state, D, chi, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol, dt=0.001, E0=nothing)

    state=normalize_IPESS_SU2_PG(state);
    #print(E0);flush(stdout);

    if E0==nothing
        E0=real(energy_CTM(D,chi,parameters, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol,state));
    end
    Bond_irrep, Bond_A_coe, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe=get_tensor_coes(state);

    #println("energy E0 is "*string(E0));flush(stdout);
    
    Bond_B_coe_tem=deepcopy(Bond_B_coe);
    Triangle_A1_coe_tem=deepcopy(Triangle_A1_coe);
    Triangle_A2_coe_tem=deepcopy(Triangle_A2_coe);

    Grad_FD=Dict([("Bond_A_coe", zeros(Float64, length(Bond_A_coe))), ("Bond_B_coe", zeros(Float64, length(Bond_B_coe))), ("Triangle_A1_coe", zeros(Float64, length(Triangle_A1_coe))),("Triangle_A2_coe", zeros(Float64, length(Triangle_A2_coe)))]);
    dE_data=[]
    Grad_FD_data=[]

    #Bond A tensor diff
    if Bond_irrep in ["A","A+iB"]
        Bond_A_grad=zeros(Float64, length(Bond_A_coe))
        for ct =1:length(Bond_A_coe)
            Bond_A_coe_tem=deepcopy(Bond_A_coe);
            Bond_A_coe_tem[ct]=Bond_A_coe_tem[ct]+dt;
            state_tem=wrap_json_state(Bond_irrep, Bond_A_coe_tem, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe);
            E=real(energy_CTM(D,chi,parameters, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol,state_tem));
            Bond_A_grad[ct]=(E-E0)/dt;
            dE_data=vcat(dE_data, E-E0);
            #print('energy is '+format(E));flush(stdout);
        end
        #print(Bond_A_grad);flush(stdout);
        Grad_FD["Bond_A_grad"]=Bond_A_grad;
        Grad_FD_data=vcat(Grad_FD_data, Bond_A_grad);
    end

    #Bond B tensor diff
    if Bond_irrep in ["B","A+iB"]
        Bond_B_grad=zeros(Float64, length(Bond_B_coe))
        for ct=1:length(Bond_B_coe)
            Bond_B_coe_tem=deepcopy(Bond_B_coe);
            Bond_B_coe_tem[ct]=Bond_B_coe_tem[ct]+dt
            state_tem=wrap_json_state(Bond_irrep, Bond_A_coe, Bond_B_coe_tem, Triangle_A1_coe, Triangle_A2_coe);
            E=real(energy_CTM(D,chi,parameters, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol,state_tem));
            Bond_B_grad[ct]=(E-E0)/dt;
            dE_data=vcat(dE_data, E-E0);
            #print('energy is '+format(E));flush(stdout);
        end
        #print(Bond_B_grad);flush(stdout);
        Grad_FD["Bond_B_grad"]=Bond_B_grad;
        Grad_FD_data=vcat(Grad_FD_data, Bond_B_grad);
    end

    #triangle A1 tensor diff
    Triangle_A1_grad=zeros(Float64, length(Triangle_A1_coe))
    for ct=1:length(Triangle_A1_coe)
        Triangle_A1_coe_tem=deepcopy(Triangle_A1_coe);
        Triangle_A1_coe_tem[ct]=Triangle_A1_coe_tem[ct]+dt
        state_tem=wrap_json_state(Bond_irrep, Bond_A_coe, Bond_B_coe, Triangle_A1_coe_tem, Triangle_A2_coe);
        E=real(energy_CTM(D,chi,parameters, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol,state_tem));
        Triangle_A1_grad[ct]=(E-E0)/dt;
        dE_data=vcat(dE_data, E-E0);
        #print('energy is '+format(E));flush(stdout);
    end
    #print(Triangle_A1_grad);flush(stdout);
    Grad_FD["Triangle_A1_grad"]=Triangle_A1_grad;
    Grad_FD_data=vcat(Grad_FD_data, Triangle_A1_grad);

    #triangle A2 tensor diff
    Triangle_A2_grad=zeros(Float64, length(Triangle_A2_coe))
    for ct=1:length(Triangle_A2_coe)
        Triangle_A2_coe_tem=deepcopy(Triangle_A2_coe);
        Triangle_A2_coe_tem[ct]=Triangle_A2_coe_tem[ct]+dt
        state_tem=wrap_json_state(Bond_irrep, Bond_A_coe, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe_tem);
        E=real(energy_CTM(D,chi,parameters, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol,state_tem));
        Triangle_A2_grad[ct]=(E-E0)/dt;
        dE_data=vcat(dE_data, E-E0);
        #print('energy is '+format(E))
    end
    #print(Triangle_A2_grad)
    Grad_FD["Triangle_A2_grad"]=Triangle_A2_grad;
    Grad_FD_data=vcat(Grad_FD_data, Triangle_A2_grad);

    # print("Energy difference is:");flush(stdout);
    # print(dE_data);flush(stdout);
    # print("Grad is:");flush(stdout);
    # print(Grad_FD_data);flush(stdout);
    # print("Normalized grad is:");flush(stdout);
    # print(Grad_FD_data/max(abs(Grad_FD_data)));flush(stdout);


    return E0,Grad_FD,Grad_FD_data

end


Grad_FiniteDiff (generic function with 3 methods)

In [10]:

function grad_line_search(state, D, chi, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol, dt, E0, grad0=None, direction0=None, alpha0=1, ls_ratio=1/3, ls_max=10,nonchiral="no")
    if nonchiral=="no"
        filenm="julia_LS_D_"*string(D)*"_chi_"*string(chi)*".json"
    elseif nonchiral=="A1_even"
        filenm="julia_LS_A1even_D_"*string(D)*"_chi_"*string(chi)*".json"
    elseif nonchiral=="A1_odd"
        filenm="julia_LS_A1odd_D_"*string(D)*"_chi_"*string(chi)*".json"
    end

    println("line search");flush(stdout);
    state=normalize_IPESS_SU2_PG(state)

    _,_,grad=Grad_FiniteDiff(state, D, chi, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol, dt, E0)
    
    println("state: "*string(get_vector(state)));flush(stdout);
    println("grad: "*string(grad));flush(stdout);

    E0=real(energy_CTM(D,chi,parameters, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol,state));
    Bond_irrep, Bond_A_coe, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe=get_tensor_coes(state)

    println("E0= "*string(E0));flush(stdout);

    direction=-grad
    #print(grad0);flush(stdout);
    #print(grad);flush(stdout);
    if grad0==nothing
        direction=-grad;
    else
        norm_grad=norm(grad)
        norm_grad0=norm(grad0)
        beta=(norm_grad^2)/(norm_grad0^2)
        direction=-grad+beta*direction0;
    end
    vec0=deepcopy(get_vector(state));
    vec_tem=[];

    #line search
    improved=false
    alpha=alpha0
    println("conjugate gradient opt");flush(stdout);
    for ls_step=1:ls_max
        vec_tem=vec0+direction*alpha*(ls_ratio^ls_step);
        state_tem=set_vector(state, vec_tem)
        println(vec_tem)
        E=real(energy_CTM(D,chi,parameters, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol,state_tem));
        println("E= "*string(E));flush(stdout);
        if E<E0
            improved=true
            break
        end
    end
    if improved
        state=set_vector(state, vec_tem)
        E=real(energy_CTM(D,chi,parameters, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol,state));
        open("filenm","w") do f
            JSON.print(f, state)
        end
    else
        println("gradient opt");flush(stdout);
        for ls_step = 1:ls_max
            vec_tem=vec0-grad*alpha*(ls_ratio^ls_step)
            state_tem=set_vector(state, vec_tem)
            E=real(energy_CTM(D,chi,parameters, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol,state_tem));
            println("E= "*string(E));;flush(stdout);
            if E<E0
                improved=true
                break
            end
        end
    
            
        if improved
            state=set_vector(state, vec_tem)
            E=real(energy_CTM(D,chi,parameters, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol,state));
            open("filenm","w") do f
                JSON.print(f, state)
            end
        else
            state=set_vector(state, vec0)
            E=E0
        end
    end
    improvement=E-E0
    
    open("filenm","w") do f
        JSON.print(f, state)
    end
    return E,state,grad,direction,improvement
end




grad_line_search (generic function with 7 methods)

In [11]:
function run_FiniteDiff(parameters,D,chi,CTM_conv_tol,CTM_ite_nums,CTM_trun_tol,Bond_irrep,init_statenm)
    
    multi_threads=true;if Threads.nthreads()==1; multi_threads=false; end
    println("number of threads: "*string(Threads.nthreads()));flush(stdout);
    
    # CTM_conv_tol=1e-6;
    # CTM_ite_nums=50;
    # CTM_trun_tol=1e-12;
   
    #init_statenm="LS_D_"*string(D)*"_chi_40.json"
    #init_statenm=nothing
    state, Bond_A_coe, Bond_B_coe, Triangle_A1_coe, Triangle_A2_coe=initial_state(Bond_irrep,D,init_statenm)

    println("optimization start");flush(stdout);
    #E0,_,_=Grad_FiniteDiff(state, cfg.ctm_args, args.chi)
    dt=0.001;
    grad0=nothing;
    direction0=nothing;
    alpha0=3;
    ls_ratio=1/3;
    ls_max=5;
    E0=nothing;
    for ite=1:100
        @time E0,state,grad,direction,improvement=grad_line_search(state,  D, chi, CTM_conv_tol,CTM_ite_nums,CTM_trun_tol, dt, E0, grad0, direction0, alpha0, ls_max)
        println("grad norm: "*string(norm(grad)));flush(stdout)
        if -improvement<1e-7
            break
        end
    end

end



run_FiniteDiff (generic function with 1 method)

In [12]:
#state_dict=read_json_state("LS_D_8_chi_40.json")
init_statenm=nothing;
CTM_conv_tol=1e-6;
CTM_ite_nums=100;
CTM_trun_tol=1e-12;
Bond_irrep="A";
run_FiniteDiff(parameters,D,chi,CTM_conv_tol,CTM_ite_nums,CTM_trun_tol,Bond_irrep,init_statenm)




number of threads: 1


Random initial state


optimization start


line search


KeyError: KeyError: key "Triangle_irrep" not found